## Dissertation Data Work - 230333550

### Hierarchy Data Work

In [56]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt

In [57]:
#Load Hierarchy Data
df = pd.read_csv("hierarchy_new_data.csv") # load raw hierarchy CSV for schema inspection and cleaning.

In [58]:
# Inspect Shape
df.shape

(32, 11)

In [59]:
# Preview
df.head(5)

,name,display_name,hierarchy_type,parent,geometry_id,portfolio_name,id,parent_id,longitude,latitude,portfolio_id
0,sbs,PES,space,sbs,23175,UoO,24052,23728,NaN,NaN,23457
1,sbs-floor-0-seminar-room-64,PES Seminar Room 9,space,sbs,23611,UoO,23961,23728,NaN,NaN,23457
2,sbs-floor-0-seminar-room-51,PES Seminar Room 2,space,sbs,23605,UoO,23955,23728,NaN,NaN,23457
3,sbs-floor-0-seminar-room-52,PES Seminar Room 3,space,sbs,23606,UoO,23956,23728,NaN,NaN,23457
4,sbs-floor-0-seminar-room-60,PES The Hive,space,sbs,23607,UoO,23957,23728,NaN,NaN,23457


### Schema and dtypes (hierarchy)
Confirm column presence and types before dropping fields.

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            32 non-null     object 
 1   display_name    32 non-null     object 
 2   hierarchy_type  32 non-null     object 
 3   parent          32 non-null     object 
 4   geometry_id     32 non-null     int64  
 5   portfolio_name  32 non-null     object 
 6   id              32 non-null     int64  
 7   parent_id       32 non-null     int64  
 8   longitude       0 non-null      float64
 9   latitude        0 non-null      float64
 10  portfolio_id    32 non-null     int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 2.9+ KB


In [61]:
df.columns

Index(['name', 'display_name', 'hierarchy_type', 'parent', 'geometry_id',
       'portfolio_name', 'id', 'parent_id', 'longitude', 'latitude',
       'portfolio_id'],
      dtype='object')

In [62]:
df.dtypes

name               object
display_name       object
hierarchy_type     object
parent             object
geometry_id         int64
portfolio_name     object
id                  int64
parent_id           int64
longitude         float64
latitude          float64
portfolio_id        int64
dtype: object

### Column Removal

The following columns were removed as they provide little or no analytical value:

- **longitude, latitude** → contain only null values.  
- **hierarchy_type, parent, portfolio_name, portfolio_id** → contain a single repeated value across all rows and therefore add no useful information for this work.

In [63]:
# Drop unused hiearchy columns
df.drop(columns=['hierarchy_type','portfolio_id', 'longitude','latitude','portfolio_name','parent'], inplace=True)

### Post-drop schema check (hierarchy)

In [64]:
# Check remaining columns
df.columns

Index(['name', 'display_name', 'geometry_id', 'id', 'parent_id'], dtype='object')

### Feature Extraction (Manual Step)

The DataFrame originally contained only:

`['name', 'display_name', 'geometry_id', 'id', 'parent_id']`

From the columns `name` and `display_name`, I manually derived the following new fields outside the notebook to better support user queries:

`['id', 'Area', 'Floor', 'Room_Name', 'geometry_id', 'parent_id']`

These fields also align with the actual **space_metadata** table structure in the database.

In [65]:
#Check for any duplicated values
df.duplicated().sum()

0

In [66]:
# Check null values
df.isnull().sum()

name            0
display_name    0
geometry_id     0
id              0
parent_id       0
dtype: int64

### Save cleaned hierarchy data

In [67]:
# Save to CSV
df.to_csv("smartviz_hierarchy_new_data_cleaned_data.csv", index=False)
print("Final enriched smartviz_hierarchy dataset saved as: smartviz_hierarchy_new_data_cleaned_data.csv")

Final enriched smartviz_hierarchy dataset saved as: smartviz_hierarchy_new_data_cleaned_data.csv


### Time-aggregated Data Work

In [68]:
# Load time-aggregated data
data = pd.read_csv('timeaggregated_new_data.csv')

### Inspect shape and columns (timeaggregated)

In [69]:
data.shape

(2411456, 14)

In [70]:
data.columns

Index(['id', 'frequency', 'start_time', 'end_time', 'metric_name',
       'aggregation', 'value', 'geometry_id', 'is_holiday', 'is_valid',
       'is_working', 'hierarchy_id', 'portfolio_id', 'data_source_id'],
      dtype='object')

### Column Removal

The following columns were removed as they provide little or no analytical value:

- **hierarchy_id** → contain only null values.  
- **portfolio_id, data_source_id** → contain a single repeated value across all rows and therefore add no useful information for this work.

In [71]:
# Drop unused timeaggregated columns
data.drop(columns=['hierarchy_id','portfolio_id','data_source_id'], inplace=True)

In [72]:
#Remaining columns
data.columns

Index(['id', 'frequency', 'start_time', 'end_time', 'metric_name',
       'aggregation', 'value', 'geometry_id', 'is_holiday', 'is_valid',
       'is_working'],
      dtype='object')

In [73]:
#Check null values in columns
print(data.isnull().sum())

id             0
frequency      0
start_time     0
end_time       0
metric_name    0
aggregation    0
value          0
geometry_id    0
is_holiday     0
is_valid       0
is_working     0
dtype: int64


### Convert start_time and end_time to UTC for consistent time operations and indexing.

In [74]:
# Force datetime with UTC handling
data['start_time'] = pd.to_datetime(data['start_time'], utc=True)
data['end_time'] = pd.to_datetime(data['end_time'], utc=True)

In [75]:
data.dtypes

id                           int64
frequency                   object
start_time     datetime64[ns, UTC]
end_time       datetime64[ns, UTC]
metric_name                 object
aggregation                 object
value                      float64
geometry_id                  int64
is_holiday                    bool
is_valid                      bool
is_working                    bool
dtype: object

In [76]:
# Check % of zero values in 'value'
zero_percentage = (data['value'] == 0).mean() * 100
print(f"Zero values in 'value': {zero_percentage:.2f}%") # Zero-value share in value

Zero values in 'value': 22.25%


In [77]:
# Create time-based features
data['hour'] = data['start_time'].dt.hour
data['dayofweek'] = data['start_time'].dt.dayofweek  # 0 = Monday
data['month'] = data['start_time'].dt.month

In [78]:
data.columns

Index(['id', 'frequency', 'start_time', 'end_time', 'metric_name',
       'aggregation', 'value', 'geometry_id', 'is_holiday', 'is_valid',
       'is_working', 'hour', 'dayofweek', 'month'],
      dtype='object')

In [79]:
data.nunique()

id             2411456
frequency            2
start_time        3072
end_time          3072
metric_name         30
aggregation          4
value            42353
geometry_id        152
is_holiday           1
is_valid             1
is_working           2
hour                24
dayofweek            7
month                5
dtype: int64

### Save Cleaned Time-aggregated data

In [80]:
# Save to CSV
data.to_csv("smartviz_timeaggregated_new_data_cleaned_data.csv", index=False)
print("Final enriched smartviz_timeaggregated dataset saved as: smartviz_timeaggregated_new_data_cleaned_data.csv")

Final enriched smartviz_timeaggregated dataset saved as: smartviz_timeaggregated_new_data_cleaned_data.csv


### Final Data Placement

After completing manual feature extraction on the hierarchy dataset, the updated file was saved in `project/data/` as **smartviz_hierarchy.csv**.  
The cleaned time-aggregated dataset was also saved in the same directory as **smartviz_occupancy_new_data.csv**.  
These files serve as the final inputs for database ingestion and chatbot integration.